In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
%matplotlib inline

In [54]:
# Data Exploration looking for data types.
test_set = pd.read_csv('statef_test.csv')
number_of_obj_type_test = []
index_of_obj_test = []
number_of_float_type_test = 0

data_types_test = test_set.dtypes
for i in range(0,100):
    
    if data_types_test[i] == 'float64':

        number_of_float_type_test= number_of_float_type_test+1
    
    elif data_types_test[i] == 'object':

        number_of_obj_type_test.append(data_types_test[i])
        index_of_obj_test.append( data_types_test.index[i])
        
        
print('Number of float64 type data:' + str(number_of_float_type_test))
print('Number of object type data:' + str(len(number_of_obj_type_test)))

Number of float64 type data:96
Number of object type data:4


In [55]:
# Training data input train data here 
def clean_statef_training (exel_file):
# Minizinging double or repeated data
# Cleaning    
    data = pd.read_csv(exel_file)
    data['x35'].replace('thurday','Thursday',inplace=True)
    data['x35'].replace('thur','Thursday',inplace=True)
    data['x35'].replace(['fri','Friday'],'Friday',inplace=True)
    data['x35'].replace('wed','Wednesday',inplace=True)
    data['x35'].replace('wednesday','Wednesday',inplace=True)
    data['x35'].replace('tuesday','Tuesday',inplace=True)
    data['x35'].replace('monday','Monday',inplace=True)
    data['x68'].replace('July','Jul',inplace=True)
    data['x68'].replace('sept.','Sep',inplace=True)
    data['x68'].replace('January','Jan',inplace=True)
    data['x68'].replace('Dev','Dec',inplace=True)
# cleaning string type to categories    
# Dealing with nan    
    data['x34']=data['x34'].str.capitalize()
    fill_values={'x34':'Other', 'x93': 'Other', 'x35':'Other', 'x68':'Unknown'}
    data.fillna(value=fill_values,inplace=True)
    data = data.copy()
    data['x34'] = data['x34'].astype('category',order = True)
    data['x34'] = data['x34'].cat.codes
    data['x68'] = data['x68'].astype('category',order = True)
    data['x68'] = data['x68'].cat.codes
    data['x35'] = data['x35'].astype('category',order = True)
    data['x35'] = data['x35'].cat.codes
    data['x93'] = data['x93'].astype('category',order = True)
    data['x93'] = data['x93'].cat.codes
#Filling missing values     
    train_set = data.apply(lambda x: x.fillna(x.mean()),axis=0)
    return train_set

In [56]:
# Testing Phase input test data here.
def clean_statef_testing (exel_file):

# Minizinging double or repeated data
# Cleaning
    test_set = pd.read_csv(exel_file)
    test_set['x35'].replace('thurday','Thursday',inplace=True)
    test_set['x35'].replace('thur','Thursday',inplace=True)
    test_set['x35'].replace(['fri','Friday'],'Friday',inplace=True)
    test_set['x35'].replace('wed','Wednesday',inplace=True)
    test_set['x35'].replace('wednesday','Wednesday',inplace=True)
    test_set['x35'].replace('tuesday','Tuesday',inplace=True)
    test_set['x35'].replace('monday','Monday',inplace=True)
    test_set['x68'].replace('July','Jul',inplace=True)
    test_set['x68'].replace('sept.','Sep',inplace=True)
    test_set['x68'].replace('January','Jan',inplace=True)
    test_set['x68'].replace('Dev','Dec',inplace=True)    
# cleaning string type to categories    
# Dealing with nan 
    test_set['x34']=test_set['x34'].str.capitalize()
    fill_values={'x34':'Other', 'x93': 'Other', 'x35':'Other', 'x68':'Unknown'}
    test_set.fillna(value=fill_values,inplace=True)
    test_set = test_set.copy()
    test_set['x34'] = test_set['x34'].astype('category',order = True)
    test_set['x34'] = test_set['x34'].cat.codes
    test_set['x68'] = test_set['x68'].astype('category',order = True)
    test_set['x68'] = test_set['x68'].cat.codes
    test_set['x35'] = test_set['x35'].astype('category',order = True)
    test_set['x35'] = test_set['x35'].cat.codes
    test_set['x93'] = test_set['x93'].astype('category',order = True)
    test_set['x93'] = test_set['x93'].cat.codes
#Filling missing values    
    testing_set = test_set.apply(lambda x: x.fillna(x.mean()),axis=0)
    return testing_set

In [57]:
# Assinging features to X and labels to y
train_data = clean_statef_training ('statef_train.csv')
test_data = clean_statef_testing('statef_test.csv')

X=train_data.drop(labels='y',axis=1)
y = train_data['y']
X_test = test_data

# Model 1 XGBOOST 

import xgboost as xgs
model3 = xgs.XGBClassifier(max_depth=11,n_estimators=300)
model3.fit(X,y)
ypred3 = model3.predict(X_test)
result1 = pd.DataFrame(model3.predict_proba(X_test)[:,1])

# Model 2 Deep-Neural Network
X_train=tf.keras.utils.normalize(np.array(X),axis=1)
y_train= np.array(y)
# Tensorflow or Deeplearning Model
X_test=tf.keras.utils.normalize(np.array(X_test),axis=1)
mod1=tf.keras.models.Sequential()
mod1.add(tf.keras.layers.Flatten())
mod1.add(tf.keras.layers.Dense(100,activation=tf.nn.relu))
mod1.add(tf.keras.layers.Dense(100,activation=tf.nn.relu))
mod1.add(tf.keras.layers.Dense(50,activation=tf.nn.relu))
#mod1.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
#mod1.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
#mod1.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
mod1.add(tf.keras.layers.Dense(2,activation=tf.nn.sigmoid))
mod1.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'],lr=0.1,)
mod1.fit(X_train,y_train,epochs=10,validation_split=0.10,batch_size=20,shuffle='Str',)
result2 = pd.DataFrame(mod1.predict_proba(X_test)[:,1])

Train on 36000 samples, validate on 4000 samples
Epoch 1/10
36000/36000 [==============================] - 5s 126us/sample - loss: 0.2488 - accuracy: 0.9016 - val_loss: 0.1332 - val_accuracy: 0.9595
Epoch 2/10
36000/36000 [==============================] - 4s 104us/sample - loss: 0.1016 - accuracy: 0.9680 - val_loss: 0.1036 - val_accuracy: 0.9682
Epoch 3/10
36000/36000 [==============================] - 4s 115us/sample - loss: 0.0729 - accuracy: 0.9787 - val_loss: 0.0790 - val_accuracy: 0.9778
Epoch 4/10
36000/36000 [==============================] - 4s 105us/sample - loss: 0.0611 - accuracy: 0.9830 - val_loss: 0.0777 - val_accuracy: 0.9808
Epoch 5/10
36000/36000 [==============================] - 4s 109us/sample - loss: 0.0548 - accuracy: 0.9846 - val_loss: 0.1351 - val_accuracy: 0.9605
Epoch 6/10
36000/36000 [==============================] - 4s 112us/sample - loss: 0.0498 - accuracy: 0.9859 - val_loss: 0.0728 - val_accuracy: 0.9822
Epoch 7/10
36000/36000 [===========================

In [58]:
result1.to_csv('results2.csv',index=False,header=False)
result2.to_csv('results1.csv',index=False,header=False)